<a href="https://colab.research.google.com/github/vsunday/gcolab/blob/main/comp_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install signate

     |████████████████████████████████| 143kB 9.0MB/s 
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=df377aa1da2b5c7eec7a57528acf6c4a3ee9415111c572aeef4294b619e1a731
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import os
os.environ['COMP_ID'] = '463'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir /root/.signate
!cp /content/drive/MyDrive/'Colab Notebooks'/signate/signate.json /root/.signate
!mkdir /content/signate
%cd /content/signate
!signate download --competition-id $COMP_ID || ls

/content/signate
sample_submit.csv

test.csv

train.csv


Download completed.


In [55]:
import pandas as pd
#import xgboost as xgb
import numpy as np

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold

In [23]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
# ss = pd.read_csv('sample_submit.csv')

In [25]:
train.head()

,id,age,sex,bmi,children,smoker,region,charges
0,0,26,male,32.665465,3,no,southeast,0
1,1,41,male,29.798725,1,no,southwest,0
2,2,28,male,32.722029,0,yes,northwest,1
3,3,20,female,38.429831,2,no,southeast,0
4,4,45,female,29.641854,1,no,northwest,0


In [69]:
def feature_engineering(data, df):
  result = data.copy(deep=True)
  # applying mean encoding to categorical features
  for c in ['sex', 'children', 'smoker', 'region']:
    result[c] = result[c].map(df.groupby(c).mean()['charges'].to_dict())

  # normalize numeric features
  for c in ['age', 'bmi']:
    result[c] = (df[c] - df[c].mean()) / df[c].std()

  return result

train_engd = feature_engineering(train, train)
train_engd.head()

,id,age,sex,bmi,children,smoker,region,charges
0,0,-0.957948,0.376932,0.041805,0.431138,0.070579,0.465686,0
1,1,0.148653,0.376932,-0.455296,0.197143,0.070579,0.245333,0
2,2,-0.810401,0.376932,0.051614,0.310345,1.182891,0.230769,1
3,3,-1.400589,0.227931,1.041364,0.386957,0.070579,0.465686,0
4,4,0.443747,0.227931,-0.482498,0.197143,0.070579,0.230769,0


In [70]:
# simple random forest classifier
X_train, X_test, y_train, y_test = train_test_split(train_engd.drop('charges',axis=1), train_engd['charges'], test_size=.1, random_state=1)
for n in range(10,210,10):
  classifier = RandomForestClassifier(n_estimators=n)
  classifier.fit(X_train, y_train)
  preds = classifier.predict(X_test)
  print(f'{n}: {sklearn.metrics.accuracy_score(y_test, preds)}')

10: 0.85
20: 0.8375
30: 0.83125
40: 0.84375
50: 0.84375
60: 0.84375
70: 0.84375
80: 0.84375
90: 0.84375
100: 0.83125
110: 0.83125
120: 0.85625
130: 0.85625
140: 0.85
150: 0.85
160: 0.85
170: 0.84375
180: 0.84375
190: 0.85
200: 0.8375


In [71]:
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_train, y_train)
preds = classifier.predict(X_test)
sklearn.metrics.accuracy_score(y_test, preds)

0.83125

In [72]:
data_test = feature_engineering(test, train)
preds_test = classifier.predict(data_test)

In [111]:
from datetime import datetime


result = pd.concat([test["id"], pd.Series(preds_test)], axis=1)

filename = f'result_{datetime.today():%Y%m%d_%H%M}.csv'
result.to_csv(filename, header=False,index=False)

#from google.colab import files
#files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [125]:
# submit
#!signate submit --competition-id $COMP_ID {filename}

You have successfully submitted your predictions.We will send you the submission result to your email address.
